In [ ]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from dotenv import load_dotenv
import os
import pandas as pd


In [ ]:
load_dotenv(r"C:\Users\kwasi\OneDrive\Documents\Personal Projects\schwab_trader\venv\.env")  # or just ".env" depending on your setup

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET")
#print(API_KEY, API_SECRET)


In [ ]:
# Initialize Alpaca historical data client (for stocks)
client = StockHistoricalDataClient(API_KEY, API_SECRET)

In [ ]:
from datetime import datetime, timedelta

def get_price_data(symbol: str, timeframe: TimeFrame = TimeFrame.Day, limit: int = 30) -> pd.DataFrame:
    """
    Fetch historical OHLCV data for a symbol using alpaca-py.

    :param symbol: Ticker symbol, e.g. "AAPL"
    :param timeframe: TimeFrame object, e.g. TimeFrame.Minute, TimeFrame.Day
    :param limit: Number of bars to fetch
    :return: pandas DataFrame with OHLCV data
    """
    end = datetime.now()
    start = end - timedelta(days=limit * 2)  # buffer for weekends/holidays

    request = StockBarsRequest(
        symbol_or_symbols=[symbol],
        timeframe=timeframe,
        start=start,
        end=end
    )

    bars = client.get_stock_bars(request).df
    symbol_df = bars[bars.index.get_level_values("symbol") == symbol].copy()
    symbol_df.index = symbol_df.index.droplevel("symbol")
    symbol_df.index = symbol_df.index.tz_convert("America/New_York")

    return symbol_df

In [ ]:
df = get_price_data("AAPL", timeframe=TimeFrame.Day, limit=30)
df[["open", "high", "low", "close", "volume"]].head() # interacting with the API and get price data

In [ ]:
from alpaca.data.requests import StockLatestQuoteRequest, StockLatestTradeRequest

# Latest quote
quote = client.get_stock_latest_quote(StockLatestQuoteRequest(symbol_or_symbols=["AAPL"]))
print(quote)

# Latest trade
trade = client.get_stock_latest_trade(StockLatestTradeRequest(symbol_or_symbols=["AAPL"]))
print(trade)

In [ ]:
from alpaca.trading.client import TradingClient

trading_client = TradingClient(API_KEY, API_SECRET, paper=True)

account = trading_client.get_account()
print(account.buying_power)
print(account.status)

In [ ]:
positions = trading_client.get_all_positions()
for p in positions:
    print(f"{p.symbol}: {p.qty} shares @ avg price {p.avg_entry_price}")

In [ ]:
orders = trading_client.get_orders()  # open, closed, all
for o in orders:
    print(f"{o.symbol} | {o.side} | {o.qty} | {o.status}")

In [ ]:
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
# placing orders 
order = MarketOrderRequest(
    symbol="AAPL",
    qty=1,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.DAY
)

response = trading_client.submit_order(order)
print(response)


In [ ]:
from alpaca.trading.client import TradingClient

calendar = trading_client.get_calendar()
for day in calendar[:5]:
    print(f"{day.date} | Open: {day.open} | Close: {day.close}")

In [ ]:
import asyncio
import nest_asyncio
from alpaca.data.live import StockDataStream
stream = StockDataStream(API_KEY, API_SECRET)

# ✅ Handler must be an async function
async def handle_quote(data):
    print("Quote:", data)

# ✅ Subscribe correctly — NOT using @decorator
stream.subscribe_quotes(handle_quote, "SPY")

# Run the stream
async def main():
    await stream.run()

nest_asyncio.apply()

asyncio.run(main())

In [ ]:
# this is a working single stock prototype
import sys
from pathlib import Path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

import os
import asyncio
import logging
import pandas as pd
from datetime import datetime, date
from collections import deque
from dotenv import load_dotenv
import requests
import json
from pytz import timezone

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.live import StockDataStream

from strategies.strategy_registry.momentum_strategy import MomentumStrategy
from core.position_sizer import DynamicPositionSizer
from indicators.atr import ATRIndicator

load_dotenv(r"C:\Users\kwasi\OneDrive\Documents\Personal Projects\schwab_trader\venv\.env")

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET")
DISCORD_WEBHOOK = os.getenv("DISCORD_WEBHOOK")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("LiveRunner")

trading_client = TradingClient(API_KEY, API_SECRET, paper=True)
stream = StockDataStream(API_KEY, API_SECRET)

symbol = "AAPL"
daily_buffer = deque(maxlen=100)
current_day_bar = None
strategy = MomentumStrategy()
sizer = DynamicPositionSizer(risk_percentage=0.07)

portfolio_history = []
total_fees = 0
peak_portfolio_value = 0
last_trade_date = None
market_closed = False

# Load historical daily bars
processed_path = Path(rf"C:\Users\kwasi\OneDrive\Documents\Personal Projects\schwab_trader\data\data_storage\proc_data\proc_{symbol}_file.json")
if processed_path.exists():
    with open(processed_path, "r") as f:
        processed_data = json.load(f)
        for item in processed_data[-100:]:
            daily_buffer.append({
                "Date": pd.to_datetime(item["Date"], unit='ms'),
                "Open": item["Open"],
                "High": item["High"],
                "Low": item["Low"],
                "Close": item["Close"],
                "Volume": item["Volume"],
                "ATR": item.get("ATR"),
                "RSI": item.get("RSI"),
                "Momentum": item.get("Momentum"),
                "VWAP": item.get("VWAP"),
                "OBV": item.get("OBV"),
                "MACD": item.get("MACD"),
                "MACD_Signal": item.get("MACD_Signal"),
                "Price_Change": item.get("Price_Change"),
                "Daily_Return": item.get("Daily_Return"),
                "Lag_Close_1": item.get("Lag_Close_1")
            })
    logger.info(f"Loaded {len(daily_buffer)} preprocessed bars for {symbol}.")
else:
    logger.warning(f"Processed data not found for {symbol}.")

def is_market_close(timestamp_utc):
    eastern = timezone("US/Eastern")
    eastern_time = timestamp_utc.astimezone(eastern)
    return eastern_time.hour == 16 and eastern_time.minute == 0

def on_market_close():
    global current_day_bar, portfolio_history, market_closed
    if not market_closed:
        logger.info("Market closed. Running end-of-day actions...")
        if current_day_bar:
            daily_buffer.append(current_day_bar)
        df = pd.DataFrame(list(daily_buffer))
        df.to_csv(f"data/processed/final_{symbol}_bars.csv", index=False)
        pd.DataFrame(portfolio_history).to_csv(f"data/processed/portfolio_{symbol}.csv", index=False)
        final_value = portfolio_history[-1]["Portfolio_Value"] if portfolio_history else 0
        drawdown = portfolio_history[-1]["Drawdown"] if portfolio_history else 0
        summary = f"📈 Market Closed\nFinal Value: ${final_value:.2f} | Drawdown: {drawdown:.2%}"
        send_discord_message(summary)
        prompt = f"The market has closed. Here's the end-of-day summary: {summary}. Provide risk commentary."
        insight = query_local_llm(prompt)
        if insight:
            send_discord_message(f"🤖 LLM Insight (EOD):\n{insight}")
        market_closed = True

async def send_heartbeat(interval_seconds=3600):
    await asyncio.sleep(10)
    send_discord_message(":rocket: LLM Trading Bot is now running.")
    while True:
        await asyncio.sleep(interval_seconds)
        send_discord_message(":heartbeat: LLM Trading Bot is still running...")

def submit_order(symbol, qty, side):
    order = MarketOrderRequest(
        symbol=symbol,
        qty=int(qty),
        side=side,
        time_in_force=TimeInForce.DAY
    )
    try:
        return trading_client.submit_order(order)
    except Exception as e:
        logger.error(f"Order submission failed: {e}")
        return None

def send_discord_message(content):
    if not DISCORD_WEBHOOK:
        return
    try:
        requests.post(DISCORD_WEBHOOK, json={"content": content})
    except Exception as e:
        logger.error(f"Failed to send Discord message: {e}")

def query_local_llm(prompt: str):
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": "deepseek", "prompt": prompt, "stream": False}
        )
        return response.json().get("response", "").strip()
    except Exception as e:
        logger.error(f"Failed to query LLM: {e}")
        return ""

async def on_bar(bar):
    global current_day_bar, peak_portfolio_value, last_trade_date

    timestamp = pd.to_datetime(bar.timestamp)
    bar_date = timestamp.date()

    if is_market_close(timestamp):
        on_market_close()

    if current_day_bar and current_day_bar['Date'].date() != bar_date:
        daily_buffer.append(current_day_bar)
        logger.info(f"Finalized bar for {current_day_bar['Date'].date()}")
        current_day_bar = None

    if current_day_bar is None:
        current_day_bar = {
            "Date": timestamp,
            "Open": bar.open,
            "High": bar.high,
            "Low": bar.low,
            "Close": bar.close,
            "Volume": bar.volume
        }
    else:
        current_day_bar["High"] = max(current_day_bar["High"], bar.high)
        current_day_bar["Low"] = min(current_day_bar["Low"], bar.low)
        current_day_bar["Close"] = bar.close
        current_day_bar["Volume"] += bar.volume

    logger.info(f"Minute update: {timestamp} | O: {bar.open:.2f} H: {bar.high:.2f} L: {bar.low:.2f} C: {bar.close:.2f} V: {bar.volume}")

    df = pd.DataFrame(list(daily_buffer) + [current_day_bar])
    if len(df) < 20:
        return

    df_tail = df.tail(20).copy()
    df_tail = ATRIndicator(df_tail).compute()
    df.update(df_tail)

    df = strategy.generate_signal(df)
    latest = df.iloc[-1]

    signal = latest['Signal']
    price = latest['Close']
    atr_value = latest['ATR']

    if pd.isna(atr_value) or atr_value <= 0:
        return

    atr_25 = df['ATR'].quantile(0.25)
    atr_75 = df['ATR'].quantile(0.75)

    market_conditions = (
        "low_volatility" if atr_value < atr_25 else
        "high_volatility" if atr_value > atr_75 else
        "normal"
    )

    try:
        account = trading_client.get_account()
        cash = float(account.cash)
    except Exception as e:
        logger.error(f"Failed to fetch account info: {e}")
        return

    try:
        position_data = trading_client.get_open_position(symbol)
        position = int(float(position_data.qty))
    except:
        position = 0

    stop_loss_price = price - (atr_value * 2)
    quantity = sizer.calculate_position_size(price, stop_loss_price, cash, market_conditions)
    trade_fee = 0.001 * price * quantity
    max_affordable_qty = cash // (price + trade_fee)
    quantity = min(quantity, max_affordable_qty)

    print("\n=== LIVE BAR UPDATE ===")
    print(df.tail(5).to_string(index=False))
    print(f"Signal: {signal} | ATR: {atr_value:.2f} | Market: {market_conditions} | Stop-loss: {stop_loss_price:.2f}")

    today_str = date.today().isoformat()
    if last_trade_date == today_str:
        return

    llm_message = None
    if signal == 1 and quantity > 0 and position == 0:
        submit_order(symbol, quantity, OrderSide.BUY)
        llm_message = f"[LIVE] BUY {quantity} {symbol} @ {price:.2f}"
        last_trade_date = today_str
    elif signal == -1 and position > 0:
        submit_order(symbol, position, OrderSide.SELL)
        llm_message = f"[LIVE] SELL {position} {symbol} @ {price:.2f}"
        last_trade_date = today_str
    elif position > 0 and price <= stop_loss_price:
        submit_order(symbol, position, OrderSide.SELL)
        llm_message = f"[LIVE] STOP-LOSS SELL {position} {symbol} @ {price:.2f}"
        last_trade_date = today_str

    if llm_message:
        logger.info(llm_message)
        prompt = f"Analyze the following trade activity and provide a short summary with risk commentary:\n{llm_message}"
        llm_response = query_local_llm(prompt)
        if llm_response:
            send_discord_message(f"🤖 LLM Insight:\n{llm_response}")

    portfolio_value = cash + position * price
    peak_portfolio_value = max(peak_portfolio_value, portfolio_value)
    drawdown = (portfolio_value - peak_portfolio_value) / peak_portfolio_value
    portfolio_history.append({
        "Date": datetime.now(),
        "Portfolio_Value": portfolio_value,
        "Cash": cash,
        "Position": position,
        "Price": price,
        "Drawdown": drawdown
    })

stream.subscribe_bars(on_bar, symbol)

stream_started = False
retry_cooldown_seconds = 300

async def run_stream_with_retry():
    global stream_started
    while not stream_started:
        try:
            logger.info("Attempting to start Alpaca stream...")
            await stream.run()
            stream_started = True
        except Exception as e:
            logger.error(f"Stream failed to start: {e}")
            send_discord_message(f"⚠️ Stream failed to start. Retrying in 5 minutes.\nError: {e}")
            await asyncio.sleep(retry_cooldown_seconds)

async def main():
    heartbeat_task = asyncio.create_task(send_heartbeat())
    await asyncio.gather(heartbeat_task, run_stream_with_retry())

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())



In [ ]:
# multi stock prototype
import sys
from pathlib import Path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

import os
import asyncio
import logging
import pandas as pd
from datetime import datetime, date
from collections import deque
from dotenv import load_dotenv
import requests
import json
from pytz import timezone

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.live import StockDataStream

from strategies.strategy_registry.momentum_strategy import MomentumStrategy
from core.position_sizer import DynamicPositionSizer
from indicators.atr import ATRIndicator

load_dotenv(r"C:\Users\kwasi\OneDrive\Documents\Personal Projects\schwab_trader\venv\.env")

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET")
DISCORD_WEBHOOK = os.getenv("DISCORD_WEBHOOK")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("LiveRunner")

trading_client = TradingClient(API_KEY, API_SECRET, paper=True)
stream = StockDataStream(API_KEY, API_SECRET)

symbols = ["AAPL", "MSFT"]

class SymbolContext:
    def __init__(self, symbol):
        self.symbol = symbol
        self.daily_buffer = deque(maxlen=100)
        self.current_day_bar = None
        self.strategy = MomentumStrategy()
        self.sizer = DynamicPositionSizer(risk_percentage=0.07)
        self.portfolio_history = []
        self.peak_portfolio_value = 0
        self.last_trade_date = None
        self.market_closed = False

        path = Path(f"data/data_storage/proc_data/proc_{symbol}_file.json")
        if path.exists():
            with open(path, "r") as f:
                for item in json.load(f)[-100:]:
                    self.daily_buffer.append({
                        "Date": pd.to_datetime(item["Date"], unit='ms'),
                        "Open": item["Open"], "High": item["High"], "Low": item["Low"],
                        "Close": item["Close"], "Volume": item["Volume"],
                        "ATR": item.get("ATR"), "RSI": item.get("RSI"), "Momentum": item.get("Momentum"),
                        "VWAP": item.get("VWAP"), "OBV": item.get("OBV"), "MACD": item.get("MACD"),
                        "MACD_Signal": item.get("MACD_Signal"), "Price_Change": item.get("Price_Change"),
                        "Daily_Return": item.get("Daily_Return"), "Lag_Close_1": item.get("Lag_Close_1")
                    })
            logger.info(f"[{symbol}] Loaded historical bars.")
        else:
            logger.warning(f"[{symbol}] Historical data not found.")

symbol_contexts = {sym: SymbolContext(sym) for sym in symbols}

def is_market_close(timestamp):
    eastern = timezone("US/Eastern")
    et = timestamp.astimezone(eastern)
    return et.hour == 16 and et.minute == 0

def send_discord_message(msg):
    if DISCORD_WEBHOOK:
        try:
            requests.post(DISCORD_WEBHOOK, json={"content": msg})
        except Exception as e:
            logger.error(f"Discord error: {e}")

def query_local_llm(prompt):
    try:
        resp = requests.post("http://localhost:11434/api/generate",
                             json={"model": "deepseek", "prompt": prompt, "stream": False})
        return resp.json().get("response", "").strip()
    except Exception as e:
        logger.error(f"LLM error: {e}")
        return ""

def submit_order(symbol, qty, side):
    try:
        return trading_client.submit_order(MarketOrderRequest(
            symbol=symbol, qty=int(qty), side=side, time_in_force=TimeInForce.DAY))
    except Exception as e:
        logger.error(f"Order error: {e}")
        return None

async def handle_bar_for_symbol(bar, ctx: SymbolContext):
    ts = pd.to_datetime(bar.timestamp)
    bar_date = ts.date()

    if is_market_close(ts) and not ctx.market_closed:
        if ctx.current_day_bar:
            ctx.daily_buffer.append(ctx.current_day_bar)
        df = pd.DataFrame(list(ctx.daily_buffer))
        df.to_csv(f"data/processed/final_{ctx.symbol}_bars.csv", index=False)
        pd.DataFrame(ctx.portfolio_history).to_csv(f"data/processed/portfolio_{ctx.symbol}.csv", index=False)
        if ctx.portfolio_history:
            pv = ctx.portfolio_history[-1]['Portfolio_Value']
            dd = ctx.portfolio_history[-1]['Drawdown']
            msg = f"📈 [{ctx.symbol}] Market closed. Value: ${pv:.2f} | Drawdown: {dd:.2%}"
            send_discord_message(msg)
            insight = query_local_llm(f"Market closed. Summary: {msg}. Risk commentary?")
            if insight:
                send_discord_message(f"🤖 LLM Insight (EOD):\n{insight}")
        ctx.market_closed = True
        return

    if ctx.current_day_bar and ctx.current_day_bar['Date'].date() != bar_date:
        ctx.daily_buffer.append(ctx.current_day_bar)
        ctx.current_day_bar = None

    if ctx.current_day_bar is None:
        ctx.current_day_bar = {"Date": ts, "Open": bar.open, "High": bar.high, "Low": bar.low, "Close": bar.close, "Volume": bar.volume}
    else:
        ctx.current_day_bar["High"] = max(ctx.current_day_bar["High"], bar.high)
        ctx.current_day_bar["Low"] = min(ctx.current_day_bar["Low"], bar.low)
        ctx.current_day_bar["Close"] = bar.close
        ctx.current_day_bar["Volume"] += bar.volume

    logger.info(f"[{ctx.symbol}] {ts} O: {bar.open} H: {bar.high} L: {bar.low} C: {bar.close} V: {bar.volume}")

    df = pd.DataFrame(list(ctx.daily_buffer) + [ctx.current_day_bar])
    if len(df) < 20:
        return
    df_tail = df.tail(20).copy()
    df_tail = ATRIndicator(df_tail).compute()
    df.update(df_tail)
    df = ctx.strategy.generate_signal(df)

    latest = df.iloc[-1]
    signal = latest['Signal']
    price = latest['Close']
    atr = latest['ATR']

    if pd.isna(atr) or atr <= 0:
        return

    atr_25, atr_75 = df['ATR'].quantile(0.25), df['ATR'].quantile(0.75)
    regime = "low_volatility" if atr < atr_25 else "high_volatility" if atr > atr_75 else "normal"

    try:
        cash = float(trading_client.get_account().cash)
    except Exception as e:
        logger.error(f"Acct fetch fail: {e}")
        return

    try:
        position = int(float(trading_client.get_open_position(ctx.symbol).qty))
    except:
        position = 0

    stop = price - (2 * atr)
    qty = ctx.sizer.calculate_position_size(price, stop, cash, regime)
    qty = min(qty, cash // (price * 1.001))

    print(f"\n=== [{ctx.symbol}] LIVE BAR ===\n{df.tail(3)}")
    print(f"Signal: {signal} | ATR: {atr:.2f} | Market: {regime} | SL: {stop:.2f}")

    today = date.today().isoformat()
    if ctx.last_trade_date == today:
        return

    if signal == 1 and qty > 0 and position == 0:
        submit_order(ctx.symbol, qty, OrderSide.BUY)
        ctx.last_trade_date = today
        send_discord_message(f"[LIVE] BUY {qty} {ctx.symbol} @ {price:.2f}")
    elif signal == -1 and position > 0:
        submit_order(ctx.symbol, position, OrderSide.SELL)
        ctx.last_trade_date = today
        send_discord_message(f"[LIVE] SELL {position} {ctx.symbol} @ {price:.2f}")
    elif position > 0 and price <= stop:
        submit_order(ctx.symbol, position, OrderSide.SELL)
        ctx.last_trade_date = today
        send_discord_message(f"[LIVE] STOP-LOSS SELL {position} {ctx.symbol} @ {price:.2f}")

    portfolio_value = cash + position * price
    ctx.peak_portfolio_value = max(ctx.peak_portfolio_value, portfolio_value)
    drawdown = (portfolio_value - ctx.peak_portfolio_value) / ctx.peak_portfolio_value
    ctx.portfolio_history.append({
        "Date": datetime.now(), "Portfolio_Value": portfolio_value, "Cash": cash,
        "Position": position, "Price": price, "Drawdown": drawdown
    })

def make_on_bar(symbol):
    async def on_bar(bar):
        await handle_bar_for_symbol(bar, symbol_contexts[symbol])
    return on_bar

for sym in symbols:
    stream.subscribe_bars(make_on_bar(sym), sym)

async def send_heartbeat():
    await asyncio.sleep(10)
    send_discord_message(":rocket: Bot running.")
    while True:
        await asyncio.sleep(3600)
        send_discord_message(":heartbeat: Bot alive.")

async def run_stream():
    while True:
        try:
            logger.info("Starting stream...")
            await stream.run()
        except Exception as e:
            logger.error(f"Stream error: {e}")
            await asyncio.sleep(300)

async def main():
    await asyncio.gather(send_heartbeat(), run_stream())

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())


In [ ]:
#modularized with symbol context for multisymbol management:
import sys
from pathlib import Path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

import os
import asyncio
import logging
import pandas as pd
from datetime import datetime, date
from collections import deque
from dotenv import load_dotenv
import requests
import json
from pytz import timezone

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.live import StockDataStream

from strategies.strategy_registry.momentum_strategy import MomentumStrategy
from strategies.strategy_registry.mean_reversion_strategy import MeanReversionStrategy
from core.position_sizer import DynamicPositionSizer
from indicators.atr import ATRIndicator

load_dotenv(r"C:\Users\kwasi\OneDrive\Documents\Personal Projects\schwab_trader\venv\.env")

API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET")
DISCORD_WEBHOOK = os.getenv("DISCORD_WEBHOOK")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("MultiSymbolLiveRunner")

trading_client = TradingClient(API_KEY, API_SECRET, paper=True)
stream = StockDataStream(API_KEY, API_SECRET)

symbol_config = {
    "AAPL": [MomentumStrategy()],
    "MSFT": [MomentumStrategy(), MeanReversionStrategy()],
}

class SymbolContext:
    def __init__(self, symbol, strategies):
        self.symbol = symbol
        self.strategies = strategies
        self.daily_buffer = deque(maxlen=100)
        self.current_day_bar = None
        self.sizer = DynamicPositionSizer(risk_percentage=0.07)
        self.portfolio_history = []
        self.peak_portfolio_value = 0
        self.last_trade_date = None
        self.market_closed = False
        self.load_historical_bars()

    def load_historical_bars(self):
        path = Path(f"data/data_storage/proc_data/proc_{self.symbol}_file.json")
        if path.exists():
            with open(path, "r") as f:
                processed_data = json.load(f)
                for item in processed_data[-100:]:
                    self.daily_buffer.append({
                        "Date": pd.to_datetime(item["Date"], unit='ms'),
                        "Open": item["Open"],
                        "High": item["High"],
                        "Low": item["Low"],
                        "Close": item["Close"],
                        "Volume": item["Volume"],
                        "ATR": item.get("ATR"),
                        "RSI": item.get("RSI"),
                        "Momentum": item.get("Momentum"),
                        "VWAP": item.get("VWAP"),
                        "OBV": item.get("OBV"),
                        "MACD": item.get("MACD"),
                        "MACD_Signal": item.get("MACD_Signal"),
                        "Price_Change": item.get("Price_Change"),
                        "Daily_Return": item.get("Daily_Return"),
                        "Lag_Close_1": item.get("Lag_Close_1")
                    })
            logger.info(f"Loaded {len(self.daily_buffer)} preprocessed bars for {self.symbol}.")
        else:
            logger.warning(f"Processed data not found for {self.symbol}.")

    def finalize_bar(self, timestamp):
        if self.current_day_bar:
            self.daily_buffer.append(self.current_day_bar)
            logger.info(f"Finalized bar for {self.symbol} on {timestamp.date()}")
            self.current_day_bar = None

    def update_bar(self, bar):
        timestamp = pd.to_datetime(bar.timestamp)
        bar_date = timestamp.date()

        if self.current_day_bar and self.current_day_bar['Date'].date() != bar_date:
            self.finalize_bar(timestamp)

        if self.current_day_bar is None:
            self.current_day_bar = {
                "Date": timestamp,
                "Open": bar.open,
                "High": bar.high,
                "Low": bar.low,
                "Close": bar.close,
                "Volume": bar.volume
            }
        else:
            self.current_day_bar["High"] = max(self.current_day_bar["High"], bar.high)
            self.current_day_bar["Low"] = min(self.current_day_bar["Low"], bar.low)
            self.current_day_bar["Close"] = bar.close
            self.current_day_bar["Volume"] += bar.volume

    def process_signal(self):
        if not self.current_day_bar:
            return

        df = pd.DataFrame(list(self.daily_buffer) + [self.current_day_bar])
        if len(df) < 20:
            return

        df_tail = df.tail(20).copy()
        df_tail = ATRIndicator(df_tail).compute()
        df.update(df_tail)

        for strategy in self.strategies:
            df = strategy.generate_signal(df)

        latest = df.iloc[-1]
        signal = latest['Signal']
        price = latest['Close']
        atr_value = latest['ATR']

        if pd.isna(atr_value) or atr_value <= 0:
            return

        atr_25 = df['ATR'].quantile(0.25)
        atr_75 = df['ATR'].quantile(0.75)

        market_conditions = (
            "low_volatility" if atr_value < atr_25 else
            "high_volatility" if atr_value > atr_75 else
            "normal"
        )

        try:
            account = trading_client.get_account()
            cash = float(account.cash)
        except Exception as e:
            logger.error(f"[{self.symbol}] Failed to fetch account info: {e}")
            return

        try:
            position_data = trading_client.get_open_position(self.symbol)
            position = int(float(position_data.qty))
        except:
            position = 0

        stop_loss_price = price - (atr_value * 2)
        quantity = self.sizer.calculate_position_size(price, stop_loss_price, cash, market_conditions)
        trade_fee = 0.001 * price * quantity
        max_affordable_qty = cash // (price + trade_fee)
        quantity = min(quantity, max_affordable_qty)

        today_str = date.today().isoformat()
        if self.last_trade_date == today_str:
            return

        msg = None
        if signal == 1 and quantity > 0 and position == 0:
            submit_order(self.symbol, quantity, OrderSide.BUY)
            msg = f"BUY {quantity} {self.symbol} @ {price:.2f}"
            self.last_trade_date = today_str
        elif signal == -1 and position > 0:
            submit_order(self.symbol, position, OrderSide.SELL)
            msg = f"SELL {position} {self.symbol} @ {price:.2f}"
            self.last_trade_date = today_str
        elif position > 0 and price <= stop_loss_price:
            submit_order(self.symbol, position, OrderSide.SELL)
            msg = f"STOP-LOSS SELL {position} {self.symbol} @ {price:.2f}"
            self.last_trade_date = today_str

        if msg:
            logger.info(msg)
            insight = query_local_llm(f"Analyze this trade: {msg}")
            send_discord_message(f"{msg}\n🤖 {insight}")

        portfolio_value = cash + position * price
        self.peak_portfolio_value = max(self.peak_portfolio_value, portfolio_value)
        drawdown = (portfolio_value - self.peak_portfolio_value) / self.peak_portfolio_value
        self.portfolio_history.append({
            "Date": datetime.now(),
            "Portfolio_Value": portfolio_value,
            "Cash": cash,
            "Position": position,
            "Price": price,
            "Drawdown": drawdown
        })

def submit_order(symbol, qty, side):
    order = MarketOrderRequest(symbol=symbol, qty=int(qty), side=side, time_in_force=TimeInForce.DAY)
    try:
        return trading_client.submit_order(order)
    except Exception as e:
        logger.error(f"Order failed: {e}")
        return None

def query_local_llm(prompt):
    try:
        res = requests.post("http://localhost:11434/api/generate", json={"model": "deepseek", "prompt": prompt, "stream": False})
        return res.json().get("response", "")
    except Exception as e:
        logger.error(f"LLM query failed: {e}")
        return ""

def send_discord_message(msg):
    if DISCORD_WEBHOOK:
        try:
            requests.post(DISCORD_WEBHOOK, json={"content": msg})
        except Exception as e:
            logger.error(f"Discord send failed: {e}")

symbol_contexts = {symbol: SymbolContext(symbol, strategies) for symbol, strategies in symbol_config.items()}

def is_market_close(timestamp_utc):
    return timestamp_utc.astimezone(timezone("US/Eastern")).hour == 16

async def on_bar(bar):
    ctx = symbol_contexts.get(bar.symbol)
    if ctx:
        ctx.update_bar(bar)
        ctx.process_signal()
        if is_market_close(pd.to_datetime(bar.timestamp)):
            ctx.finalize_bar(pd.to_datetime(bar.timestamp))

for symbol in symbol_contexts:
    stream.subscribe_bars(on_bar, symbol)

async def send_heartbeat():
    await asyncio.sleep(5)
    send_discord_message(":rocket: Bot running with multi-symbol support")
    while True:
        await asyncio.sleep(3600)
        send_discord_message(":heartbeat: Still running...")

async def run():
    await asyncio.gather(
        send_heartbeat(),
        stream.run()
    )

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(run())
